In [1]:
import glob
import pathlib

import polars as pl

In [31]:
files = list(pathlib.Path.home().joinpath('tmp', 'taxi').glob('*.parquet'))

In [32]:
%%time

df = pl.scan_parquet(files)

CPU times: user 200 μs, sys: 218 μs, total: 418 μs
Wall time: 446 μs


In [38]:
%%time

df = pl.concat(
    [pl.scan_parquet(file) for file in files]
)

CPU times: user 595 μs, sys: 442 μs, total: 1.04 ms
Wall time: 1.15 ms


In [39]:
%%time

df.collect().shape

CPU times: user 19.7 s, sys: 4.35 s, total: 24 s
Wall time: 4.78 s


(179807942, 19)

In [40]:
%%time

df.collect().estimated_size('gb')

CPU times: user 19.7 s, sys: 4.06 s, total: 23.7 s
Wall time: 4.87 s


24.36149333976209

In [41]:
%%time

df.collect().head()

CPU times: user 19.7 s, sys: 3.92 s, total: 23.6 s
Wall time: 4.63 s


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2022-10-01 00:03:41,2022-10-01 00:18:39,1.0,1.7,1.0,"""N""",249,107,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5,0.0
2,2022-10-01 00:14:30,2022-10-01 00:19:48,2.0,0.72,1.0,"""N""",151,238,2,5.5,0.5,0.5,0.0,0.0,0.3,9.3,2.5,0.0
2,2022-10-01 00:27:13,2022-10-01 00:37:41,1.0,1.74,1.0,"""N""",238,166,1,9.0,0.5,0.5,2.06,0.0,0.3,12.36,0.0,0.0
1,2022-10-01 00:32:53,2022-10-01 00:38:55,0.0,1.3,1.0,"""N""",142,239,1,6.5,3.0,0.5,2.05,0.0,0.3,12.35,2.5,0.0
1,2022-10-01 00:44:55,2022-10-01 00:50:21,0.0,1.0,1.0,"""N""",238,166,1,6.0,0.5,0.5,1.8,0.0,0.3,9.1,0.0,0.0


In [42]:
%%time

(
    df.with_columns(
        is_congested=pl.col('congestion_surcharge').fill_null(0) > 0
    )
    .group_by('DOLocationID')
    .agg(
        mean_fare=pl.mean('fare_amount'),
        count=pl.len(),
        congestion=pl.mean('is_congested')
    )
    .sort('count', descending=True)
    .collect()
)

CPU times: user 9.7 s, sys: 1.86 s, total: 11.6 s
Wall time: 2.35 s


DOLocationID,mean_fare,count,congestion
i64,f64,u32,f64
236,9.567141,7687524,0.930577
237,9.27386,7242212,0.941435
161,11.019058,6555290,0.935838
170,10.79023,5473077,0.932653
162,-15.278814,5045552,0.935021
…,…,…,…
2,40.340785,242,0.090909
105,16.492748,131,0.740458
199,12.733333,15,0.266667


In [8]:
df.collect().shape

(36256549, 19)

In [9]:
df.collect().estimated_size('gb')

4.911875405348837